In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import *
from sklearn.naive_bayes import GaussianNB
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import balanced_accuracy_score
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob
from imblearn.over_sampling import (RandomOverSampler, SMOTENC)
from sklearn.linear_model import (ElasticNet, Ridge, Lasso, LogisticRegression)
from sklearn.ensemble import (RandomForestRegressor, RandomForestClassifier)
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.svm import SVR
from sklearn.model_selection import RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import json
from category_encoders import *

In [3]:
models_dct={}

In [4]:
def get_training_data(X,Y):
    
    X_train, X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.20, random_state=42)
    print('X Train',X_train.shape)
    print('X test',X_test.shape)
    print('Y Train',Y_train.shape)
    print('Y test',Y_test.shape)
    #X_train,Y_train = train[:,0:cols-2],train[:,cols-1]
    #X_test,Y_test = test[:,0:cols-2],test[:,cols-1]
    return X_train,Y_train,X_test,Y_test

In [5]:
def classification_metrics(Y_pred,Y_true):
    cnf_matrix = metrics.confusion_matrix(Y_true, Y_pred)
    accuracy=metrics.accuracy_score(Y_true, Y_pred)
    bal_accuracy= metrics.balanced_accuracy_score(Y_true, Y_pred)
    precision=metrics.precision_score(Y_true, Y_pred,average='weighted')
    recall=metrics.recall_score(Y_true, Y_pred,average='weighted')
    fscore=metrics.f1_score(Y_true, Y_pred,average='weighted')
    #auc = metrics.roc_auc_score(Y_true, Y_pred,average='micro')
    return accuracy,precision,recall,fscore,bal_accuracy

In [6]:
def display_metrics(classifierName,Y_pred,Y_true):
    print("______________________________________________")
    print(("Classifier: "+classifierName))
    if np.array_equal(Y_pred,Y_true):
        print("Match")
        
    acc, precision, recall, f1score,bal_accuracy = classification_metrics(Y_pred,Y_true)
    mc_points = Y_pred==Y_true
    mc_p=np.size(mc_points) - np.count_nonzero(mc_points)
    print(("Accuracy: "+str(acc)))
    #print(("AUC: "+str(auc_)))
    print(("Precision: "+str(precision)))
    print(("Recall: "+str(recall)))
    print(("F1-score: "+str(f1score)))
    print(("Balanced Accuracy: "+str(bal_accuracy)))
    #print(("Misclassified Points: "+str(mc_p)))
    
    print("______________________________________________")
    print("")
    
    return acc

In [7]:
def AlphaGridSearch(X_train,Y_train):
    print('Performing Grid Searching for best alpha...')
    cvLasso = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
    lasso = Lasso(max_iter = 1e5, normalize = True)
    grid = dict()
    grid['alpha'] = [1e-5, 1e-3, 1e-2, 1, 10]
    # define search
    search = GridSearchCV(lasso, grid, scoring='neg_mean_absolute_error', cv=cvLasso, n_jobs=-1)
    # perform the search
    results = search.fit(X_train, Y_train)
    alpha = results.best_params_['alpha']
    return alpha

def LGGridSearch(X_train,Y_train):
    print('Performing Grid Searching for best gamma and labmda...')
    cvEnet = RepeatedKFold(n_splits=5, n_repeats = 3, random_state = 1)
    enet = ElasticNet(max_iter = 1e3, normalize = True)
    # define grid
    grid = dict()
    grid['alpha'] = [1e-2, 1e-1, 1.0, 10.0]
    grid['l1_ratio'] = [0, 0.5, 1.]
    search = GridSearchCV(enet, grid, scoring='neg_mean_absolute_error', cv=cvEnet, n_jobs=-1)
    # perform the search
    results_enet = search.fit(X_train, Y_train)
    
    lambda_enet = results_enet.best_params_['alpha']
    gamma_enet = results_enet.best_params_['l1_ratio']

    return lambda_enet, gamma_enet


In [8]:
def LassoVarSelection(X_train,Y_train,X_test,Y_test):
    alpha=AlphaGridSearch(X_train,Y_train)
    lasso = Lasso(alpha = alpha, random_state = 42)
    lasso.fit(X_train, Y_train)
    cols_lasso = []
    weight_lasso = []
    print(lasso.coef_)
    for feature, weight in zip(X_train.columns.values, lasso.coef_):
        if weight != 0:
            cols_lasso.append(feature)
            weight_lasso.append(weight)
    
    print(cols_lasso)
    
    train_X_lasso = X_train[cols_lasso]
    test_X_lasso = X_test[cols_lasso]
    return train_X_lasso,test_X_lasso

In [9]:
def RidgeVarSelection(X_train, Y_train, X_test, Y_test):
      alpha = AlphaGridSearch(X_train, Y_train)
      ridge = Ridge(alpha = alpha, random_state = 42)
      ridge.fit(X_train, Y_train)
      cols_ridge = []
      weight_ridge = []
      for feature, weight in zip(X_train.columns.values, ridge.coef_):
        if weight != 0:
            cols_ridge.append(feature)
            weight_ridge.append(weight)
            
      print(cols_ridge)
      train_X_ridge = X_train[cols_ridge]
      test_X_ridge = X_test[cols_ridge]
      return train_X_ridge, test_X_ridge


In [10]:
def ElasticNetVarSelection(X_train, Y_train, X_test, Y_test):
    lambda_enet, gamma_enet = LGGridSearch(X_train, Y_train)
    e_net = ElasticNet(alpha = lambda_enet, l1_ratio = gamma_enet)
    e_net.fit(X_train, Y_train)
    # calculate the prediction and mean square error
    cols_enet = []
    weight_enet = []
    for feature, weight in zip(X_train.columns.values, e_net.coef_):
        if weight != 0:
            cols_enet.append(feature)
            weight_enet.append(weight)
    
    train_X_enet = X_train[cols_enet]
    test_X_enet = X_test[cols_enet]

    return train_X_enet, test_X_enet

In [11]:
def RandomForest(X_train, Y_train, X_test, Y_test, param_grid, airport_nm):
    
    train_X_lasso,test_X_lasso=LassoVarSelection(X_train,Y_train,X_test,Y_test)
    train_X_ridge,test_X_ridge=RidgeVarSelection(X_train,Y_train,X_test,Y_test)
    train_X_enet,test_X_enet=ElasticNetVarSelection(X_train,Y_train,X_test,Y_test)
    
    rf = RandomForestClassifier()
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = 2, verbose = 2)
    
    
    # Lasso with RF
    grid_search.fit(train_X_lasso, Y_train)
    rf_lasso = RandomForestClassifier(**grid_search.best_params_, oob_score = True)
    rf_lasso.fit(train_X_lasso, Y_train)
    rf_lasso_pred = rf_lasso.predict(test_X_lasso)
    rf_lasso_prob = rf_lasso.predict_proba(test_X_lasso)
    lld_lasso = np.sum(rf_lasso_prob[:,4:].mean(axis = 0))
    filename = 'rf_lasso_{}.sav'.format(airport_nm)
    pickle.dump(rf_lasso, open(filename, 'wb'))
    accuracy= display_metrics(filename, rf_lasso_pred,Y_test)
    models_dct[filename]={'airport':airport_nm, 'features':list(train_X_lasso.columns), 'accuracy': accuracy, 'LL Delayed':lld_lasso}
    
    # Ridge with RF
    rf_ridge = RandomForestClassifier(**grid_search.best_params_, oob_score = True)
    rf_ridge.fit(train_X_ridge, Y_train)
    rf_ridge_pred = rf_ridge.predict(test_X_ridge)
    rf_ridge_prob = rf_ridge.predict_proba(test_X_ridge)
    lld_ridge = np.sum(rf_ridge_prob[:,4:].mean(axis = 0))
    filename = 'rf_ridge_{}.sav'.format(airport_nm)
    pickle.dump(rf_ridge, open(filename, 'wb'))
    accuracy= display_metrics(filename, rf_ridge_pred,Y_test)
    models_dct[filename]={'airport':airport_nm, 'features':list(train_X_ridge.columns), 'accuracy': accuracy, 'LL Delayed':lld_ridge}
    
    # Enet with RF
    rf_enet = RandomForestClassifier(**grid_search.best_params_, oob_score = True)
    rf_enet.fit(train_X_enet, Y_train)
    rf_enet_pred = rf_enet.predict(test_X_enet)
    rf_enet_prob = rf_enet.predict_proba(test_X_enet)
    lld_enet = np.sum(rf_enet_prob[:,4:].mean(axis = 0))
    filename = 'rf_enet_{}.sav'.format(airport_nm)
    pickle.dump(rf_enet, open(filename, 'wb'))
    accuracy= display_metrics(filename, rf_enet_pred,Y_test)
    models_dct[filename]={'airport':airport_nm, 'features':list(train_X_enet.columns), 'accuracy': accuracy, 'LL Delayed':lld_enet}
    return models_dct

In [12]:
import os
import datetime as dt
def combine_files():
    filePath= '/Users/jinhaenglee/Desktop/Georgia_Tech_OMSA/Practicum/Data/'
    i=0
    dct={}
    for file in os.listdir(filePath):
        f=file
        if f.endswith('.xlsx'):
            tmpDF=pd.read_excel(open(filePath+'/'+f, 'rb'),sheet_name='Data',header=1)
            tmpDF['airportNM']=f[0:3]
            dct[f]=tmpDF
    
    df_lst=[]
    for k,v in dct.items():
        df_lst.append(v)
    dataDF= pd.concat(df_lst)
    dataDF.rename(columns={dataDF.columns[15]: 'WXCodes'},inplace=True)
    dataDF.rename(columns={dataDF.columns[22]: 'SchedDepart'},inplace=True)
    dataDF.rename(columns={dataDF.columns[23]: 'SchedArriv'},inplace=True)
    
    return dataDF


In [13]:
def dataprep(df,airport):
    print(df.columns)
    #new_header = df.iloc[0]
    #df = df[1:].reset_index(drop = True)
    #df.columns = new_header
    #.rename(columns = df.iloc[0])
    y = df['Avg Gate Arrival Delay']
    df = df.iloc[:, :24] ## Take Columns A-X
    df['Avg Gate Arrival Delay'] = y
    df = df.rename(columns = {'WX\nCodes':'WXCodes', 'Sched.\nDepart': 'SchedDepartures', 'Sched.\nArriv': 'SchedArriavls'})
    ## Removing Nulls

    # It would make sense to remove null rows for Sced. Departures and Sched Arrivals as we Can't really replace them with anything.
    
    df = df[~df['SchedDepart'].isnull()]

    #1. There are common null values for METAR data. Only 24 hour of data provided for METAR. Remove these rows after midnight
    # here I am taking Vis column but any other METAR column shares same NaNs after midnight.
    df = df[~df['Vis'].isnull()]

    # Sky 1 NaN values 
    df = df[~df['Sky 1'].isnull()]

    # Dealing with NAN
    # Recall Wind Gust Finite values are excessive wind speed from Col F. Null values are steady state. Fillna with zeros as there are no excessive wind
    df['Wind Gust'] = df['Wind Gust'].fillna(0)

    # Similarly Peak Gust Speed's NaN can be treated as 0.
    df['Peak Gust Speed'] = df['Peak Gust Speed'].fillna(0)

    # Layer 1 is NaN if Sky 1 is CLR (No Clouds below 12000 ft). Let's fill na with max(Layer 1)
    df['Layer 1'].fillna(df['Layer 1'].max(), inplace = True)

    df['Ice 1 Hour'].fillna(0, inplace = True)
    df['Ice 3 Hour'].fillna(0, inplace = True)
    df['Ice 6 Hour'].fillna(0, inplace = True)

    # drop unrelevant columns
    df.drop(columns = ['Time', 'Layer 2', 'Layer 3', 'Sky 2', 'Sky 3', 'Feels Like', 'Ice 1 Hour', 'Ice 3 Hour', 'Ice 6 Hour'], inplace = True)

    # Deal with nulls in Sky 1
    # Deal with WxCodes
    df['WXCodes'].fillna('Regular', inplace = True)

    # Converting Objects to Floats
    for col in df.columns:
        if col != 'Sky 1' and col != 'WXCodes' and col != 'Peak Gust Direction':
            df[col] = df[col].astype(float)




    # Save response Variable seperately and remove from dataframe
    df['y'] = df['Avg Gate Arrival Delay'].div(10).apply(np.floor)
    y = df['y']
    #print(y)
    del df['Avg Gate Arrival Delay']
    del df['y']

    # Bucketting Wind Direction and Peak Gust WindDirection

    df.loc[(df['Peak Gust Direction'] >= 0) & (df['Peak Gust Direction'] < 11.25),'PGDirectionBkt' ]='N'
    df.loc[(df['Peak Gust Direction'] >= 11.25) & (df['Peak Gust Direction'] < 33.75),'PGDirectionBkt' ]='NNE'
    df.loc[(df['Peak Gust Direction'] >= 33.75) & (df['Peak Gust Direction'] < 56.25),'PGDirectionBkt' ]='NE'
    df.loc[(df['Peak Gust Direction'] >= 56.25) & (df['Peak Gust Direction'] < 78.75),'PGDirectionBkt' ]='ENE'
    df.loc[(df['Peak Gust Direction'] >= 78.75) & (df['Peak Gust Direction'] < 101.25),'PGDirectionBkt' ]='E'
    df.loc[(df['Peak Gust Direction'] >= 101.25) & (df['Peak Gust Direction'] < 121.75),'PGDirectionBkt' ]='ESE'
    df.loc[(df['Peak Gust Direction'] >= 123.75) & (df['Peak Gust Direction'] < 146.25),'PGDirectionBkt' ]='SE'
    df.loc[(df['Peak Gust Direction'] >= 146.25) & (df['Peak Gust Direction'] < 168.75),'PGDirectionBkt' ]='SSE'
    df.loc[(df['Peak Gust Direction'] >= 168.75) & (df['Peak Gust Direction'] < 191.25),'PGDirectionBkt' ]='S'
    df.loc[(df['Peak Gust Direction'] >= 191.25) & (df['Peak Gust Direction'] < 213.75),'PGDirectionBkt' ]='SSW'
    df.loc[(df['Peak Gust Direction'] >= 213.75) & (df['Peak Gust Direction'] < 236.25),'PGDirectionBkt' ]='SW'
    df.loc[(df['Peak Gust Direction'] >= 236.25) & (df['Peak Gust Direction'] < 258.75),'PGDirectionBkt' ]='WSW'
    df.loc[(df['Peak Gust Direction'] >= 258.75) & (df['Peak Gust Direction'] < 281.25),'PGDirectionBkt' ]='W'
    df.loc[(df['Peak Gust Direction'] >= 281.25) & (df['Peak Gust Direction'] < 303.75),'PGDirectionBkt' ]='WNW'
    df.loc[(df['Peak Gust Direction'] >= 303.75) & (df['Peak Gust Direction'] < 326.25),'PGDirectionBkt' ]='NW'
    df.loc[(df['Peak Gust Direction'] >= 326.25) & (df['Peak Gust Direction'] < 348.75),'PGDirectionBkt' ]='NNW'
    df.loc[(df['Peak Gust Direction'] >= 348.75) & (df['Peak Gust Direction'] < 360),'PGDirectionBkt' ]='N'

    # Deal with Peak Gust Direction
    df['PGDirectionBkt']=df['PGDirectionBkt'].fillna('NA')
    del df['Peak Gust Direction']
    
    

    finalDf = df.copy()
    pgdir_dct={}
    
    # Converting Objects to Floats
    #for col in finalDf.columns:
     #   finalDf[col] = finalDf[col].astype(float)
     
    continuous_columns=['Temp', 'Dew', 'T/D Spread', 'Wind Direction', 
                        'Wind Speed','Wind Gust', 'Altimeter', 'Vis', 'Layer 1', 'Peak Gust Speed', 'SchedDepart','SchedArriv']
    
    
    #categorical_columns=['Sky 1', 'WXCodes', 'PGDirectionBkt']
    
    print("******************************Input to Encoder******************************")
    print(finalDf.columns)
    
    encoder = BinaryEncoder().fit(finalDf)
    #encoder = OneHotEncoder().fit(finalDf)
    finalDf_enc=encoder.transform(finalDf)
    print("******************************Output of Encoder******************************")
    print(finalDf_enc.columns)
    
    
    categorical_columns=[col for col in finalDf_enc.columns if col not in continuous_columns]
    
    df_corr = finalDf_enc[continuous_columns].corr(method='pearson', min_periods=1)
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > 0.8).any()
    #print(df_not_correlated.columns)
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    fnl_col_lst=list(un_corr_idx)
    fnl_col_lst+=categorical_columns
    finalDf_enc = finalDf_enc[fnl_col_lst]
    
    
   
    print(finalDf_enc.columns)
    filename = 'encoder_{}.sav'.format(airport)
    pickle.dump(encoder, open(filename, 'wb'))
    

    
    '''
    pgdl = finalDf['PGDirectionBkt'].drop_duplicates()
    for i,d in enumerate(pgdl):
        pgdir_dct[d]=i
    finalDf.replace({"PGDirectionBkt": pgdir_dct},inplace=True)
    WXDir={}
    wxl = finalDf['WXCodes'].drop_duplicates()
    for i,d in enumerate(wxl):
        WXDir[d]=i
    finalDf.replace({"WXCodes": WXDir},inplace=True)

    # Get dummies for Sky 1
    dummy = pd.get_dummies(finalDf['Sky 1'])
    finalDf = pd.concat([finalDf, dummy], axis = 1)
    del finalDf['Sky 1']
    '''

    
    X = finalDf_enc.copy()    
    print(X.columns)
    
    
    # Bucket response Variable into intervals of 10 minutes
    #bi_y = np.zeros((len(y),1))
    #intervals = np.arange(10, y.max(), 10)
    #for i in range(1, len(intervals)):
    #    bi_y[(y > intervals[i]) & (y <= intervals[i] + 10)] = i
    
    

    

    # OverSample due to Imbalance Data
    
    
    #y = preprocessing.label_binarize(y)
    #print(y)
    #y = LabelEncoder().fit_transform(y)
    # transform the dataset
    #oversample = SMOTE()
    #X_ros, y_ros = oversample.fit_resample(X, y)
    
    ros = RandomOverSampler(random_state= 42)

    # fit predictor and target variable
    X_ros, y_ros = ros.fit_resample(X, y)

    print('Original dataset shape', len(y))
    print('Resample dataset shape', len(y_ros))

    # Split the data into training and testing
    finalDf = pd.DataFrame(X_ros, columns = X.columns)
    
    
    train_X, test_X, train_y, test_y = train_test_split(X_ros, y_ros, test_size = 0.2, random_state = 42)
    
    return train_X, test_X, train_y, test_y

    
    

In [14]:
def SelectBestToJson(models_dct, airport_nm):
    filtered_d = {k:v for (k,v) in models_dct.items() if airport_nm in v['airport']}
    a =sorted(filtered_d.items(),key=lambda x: x[1]['accuracy'],reverse=True)[:1]
    dict(a)
    return a

In [15]:
def train(flag):
    
    param_grid = {'bootstrap': [True],
                   'max_depth': [50,70,100],
                    'max_features': [5],
                    'min_samples_leaf': [3],
                    'min_samples_split': [8],
                    'n_estimators': [50, 100, 200]}
    

        
    dataDF = combine_files()
    if flag == 0:
      airports=list(dataDF['airportNM'].unique())    
    elif flag == 1:
      airports = ['ATL']
    
    
    for airport in airports:
        print('Starting data prep for',airport)
        airportdf=dataDF[(dataDF['airportNM']==airport)]
        train_X, test_X, train_y, test_y = dataprep(airportdf,airport)
        print('Finished data prep for',airport)
        models_dct = RandomForest(train_X, train_y,test_X,test_y,param_grid,airport)
        finalModels = SelectBestToJson(models_dct, airport)
        print('Model training completed')
        
    
        filename = 'rf_{}.json'.format(airport)
        with open(filename, "w") as outfile:
            json.dump(finalModels, outfile)
    

In [16]:
def main():
    trainFlg='Y'
    if trainFlg=='Y':
        train(1)
        
if __name__ == "__main__":
    #weatherDataDF=combine_files()
    main()


Starting data prep for ATL
Index(['Time', 'Temp', 'Dew', 'T/D Spread', 'Wind Direction', 'Wind Speed',
       'Wind Gust', 'Altimeter', 'Vis', 'Sky 1', 'Sky 2', 'Sky 3', 'Layer 1',
       'Layer 2', 'Layer 3', 'WXCodes', 'Ice 1 Hour', 'Ice 3 Hour',
       'Ice 6 Hour', 'Peak Gust Speed', 'Peak Gust Direction', 'Feels Like',
       'SchedDepart', 'SchedArriv', 'Actual\nDepartures', 'Actual\nArrivals',
       '% On-Time Gate Departures', '% On-Time Airport Departures',
       '% On-Time Gate Arrivals', 'Avg Gate Departure Delay',
       'Avg Taxi Out Time', 'Avg Taxi Out Delay', 'AvgAirport Departure Delay',
       'Avg Airborne Delay', 'Avg Taxi In Delay', 'Avg Block Delay',
       'Avg Gate Arrival Delay', 'Ceiling', 'Weather Missing',
       'Perform Missing', 'Either Missing', 'Good Weather', 'Delayed <XX min',
       'Over\nNight', 'Ignore', 'Speed', 'Gust', 'Vis.1', 'Ceil', 'Comb',
       'airportNM'],
      dtype='object')
******************************Input to Encoder************

/Users/jinhaenglee/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Original dataset shape 8658
Resample dataset shape 126702
Finished data prep for ATL
Performing Grid Searching for best alpha...
[ 2.58385567e-01 -1.61607554e-01  6.40568840e-03 -3.22222941e-01
  4.69740575e-02  1.11186769e+01 -3.00735010e-01 -6.05204164e-07
 -7.14448776e-02 -1.48583244e-01  0.00000000e+00 -2.77622485e+00
 -2.16058050e+00 -4.41347751e+00  0.00000000e+00  4.50711313e+00
  9.18650089e-01 -7.28595601e-01  7.85533086e-02  7.49239459e-01
  0.00000000e+00  9.48452900e-01 -1.31356419e+00  3.03846992e+00
 -2.10269636e+00]
['Temp', 'T/D Spread', 'Wind Direction', 'Wind Speed', 'Wind Gust', 'Altimeter', 'Vis', 'Layer 1', 'Peak Gust Speed', 'SchedDepart', 'Sky 1_1', 'Sky 1_2', 'Sky 1_3', 'WXCodes_1', 'WXCodes_2', 'WXCodes_3', 'WXCodes_4', 'WXCodes_5', 'PGDirectionBkt_1', 'PGDirectionBkt_2', 'PGDirectionBkt_3', 'PGDirectionBkt_4']
Performing Grid Searching for best alpha...


/Users/jinhaenglee/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.48632e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


['Temp', 'T/D Spread', 'Wind Direction', 'Wind Speed', 'Wind Gust', 'Altimeter', 'Vis', 'Layer 1', 'Peak Gust Speed', 'SchedDepart', 'Sky 1_1', 'Sky 1_2', 'Sky 1_3', 'WXCodes_1', 'WXCodes_2', 'WXCodes_3', 'WXCodes_4', 'WXCodes_5', 'PGDirectionBkt_1', 'PGDirectionBkt_2', 'PGDirectionBkt_3', 'PGDirectionBkt_4']
Performing Grid Searching for best gamma and labmda...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
______________________________________________
Classifier: rf_lasso_ATL.sav
Accuracy: 0.9917919576970128
Precision: 0.9919903687229613
Recall: 0.9917919576970128
F1-score: 0.9916881562123525
Balanced Accuracy: 0.991922722773933
______________________________________________

______________________________________________
Classifier: rf_ridge_ATL.sav
Accuracy: 0.991831419438854
Precision: 0.9921395852454198
Recall: 0.991831419438854
F1-score: 0.9917309937563891
Balanced Accuracy: 0.9919597088211967
______________________________________________

_______________________